In [1]:
from datetime import datetime,timedelta,date
import time
from pandas.tseries.offsets import BDay
import numpy as np
import pandas as pd
import sqlite3

In [22]:
def checkScore(df,selecttype='系統-決策樹',checktype='days',caldays=5,endday= ''):    
   
    checkdf = df.copy()
    #checkdf = checkdf[(checkdf['date'] >= '2018-04-21') & (checkdf['股票代號'] == 2501)]
    #checkdf = checkdf[(checkdf['日期'] >= '2018-04-21')]
    selecttype = selecttype
    checktype = checktype # days,sell,today
    print('checktype:',checktype)
    caldays = caldays
    
    print('caldays:',caldays)
    
    global dfinv
    global dfprice
    dfprice1 = pd.DataFrame()
    
    if endday == '':
        endday = datetime.strftime(datetime.now(),'%Y-%m-%d')
        
    today = datetime.strptime(endday,'%Y-%m-%d')

    dateformat = '%Y-%m-%d'
    colNames = ['選股類型','選股日','股票代號','股票名稱','收盤價','買入日','買入收盤價'
                ,'賣出日','賣出收盤價','漲幅','賺','計算類型','計算天數']

    checklist = []

    for index, row in checkdf.iterrows():
        selltype = checktype
        checkdit = {}

        stockno = row['證券代號']
        stockno = str(stockno)
        #print('stockno',stockno)

        stockname = row['證券名稱']
        #print('stockname',stockname)

        #price = row['收盤價']
        #if type(price) is str:
        #    price = float(price.replace(",",''))

        #取得系統選股的日期
        selectdate = row['交易日']

        #取得系統選股的日期,轉成日期格式
        selectdate = datetime.strptime(selectdate,dateformat)
        selectdatef = selectdate.strftime(dateformat)
        #print('selectdatef',selectdatef)

         #找出選股日的價格
        selectdf = dfprice[(dfprice['交易日'] == selectdatef) & (dfprice['證券代號'] == stockno)]
        if len(selectdf) > 0 :
            price = selectdf.iloc[0,3]
            if (type(price) is str) :
                if price != '--' :
                    price = float(price.replace(",",''))
                else:
                    price = 'not found'
        else:
            price = 'not found'

        #取得買入日
        #buydate = selectdate + BDay(1)
        buydate = np.busday_offset(selectdate, 1, roll='forward',weekmask=weekmask,holidays=holidays)
        buydate = buydate.astype(datetime)
        buydatef = buydate.strftime(dateformat)
        #print('buydatef',buydatef)
        selldatef = ""
        
        #取得賣出日
        #selldate5 = buydate + BDay(5)
        if checktype == 'today':
            selldate = today
        elif checktype == 'sell':
            #從放空的檔案找
            emptydf = dfempty[(dfempty['日期'] > buydatef) & (dfempty['股票代號'] == stockno)]
            if len(emptydf) > 0 :
                emptydate = emptydf.iloc[0,0]
                #print('emptydate:',emptydate)

                selldate = np.busday_offset(emptydate, 1, roll='forward',weekmask=weekmask,holidays=holidays)
                selldate = selldate.astype(datetime)
            else:
                selldate = today
                selltype = 'sell'
        elif checktype == 'max':
            dfprice1 = dfprice.loc[(dfprice['交易日'] > buydatef) & (dfprice['證券代號'] == stockno),:]
            x=dfprice1.max()
            
            selldatef = x[0]
        else:
            selldate = np.busday_offset(buydatef, caldays, roll='forward',weekmask=weekmask,holidays=holidays)
            selldate = selldate.astype(datetime)
        
        if checktype != 'max':
            selldatef = selldate.strftime(dateformat)
            #print('selldatef',selldatef)

        #找出買入日的價格
        buydf = dfprice[(dfprice['交易日'] == buydatef) & (dfprice['證券代號'] == stockno)]
        if len(buydf) > 0 :
            buyprice = buydf.iloc[0,3]
            if (type(buyprice) is str) :
                if buyprice != '--' :
                    buyprice = float(buyprice.replace(",",''))
                else:
                    buyprice = 'not found'
        else:
            buyprice = 'not found'

         #找出賣出日的價格
        selldf = dfprice[(dfprice['交易日'] == selldatef) & (dfprice['證券代號'] == stockno)]
        if len(selldf) > 0 :
            sellprice = selldf.iloc[0,3]
            if type(sellprice) is str:
                if sellprice != '--' :
                    sellprice = float(sellprice.replace(",",''))                    
                else:
                    sellprice = 'not found'
        else:
            sellprice = 'not found'
        #print('check:',stockno,stockname,buydatef,buyprice,selldatef,sellprice)
        
       
        if  buyprice == 'not found' or sellprice == 'not found' :
            diff = 'not found'
            updown = 'not found'
            updatecaldays = 0
        else :
            #計算N日漲幅
            diff = round(((sellprice - buyprice)/buyprice)*100,2)

            if diff >= 1:
                updown = 'Y'
            else:
                updown = 'N'
            #print(buyprice,sellprice,diff,updown)
            caldays = np.busday_count(begindates=buydatef, enddates=selldatef,weekmask=weekmask,holidays=holidays)
            updatecaldays = caldays
            
        if checktype != 'days':
            #caltype = '%s-從買股日至 %s 日' % (selltype,selldatef)
            caltype = '%s' % (selltype)
        else:
            caltype =str(caldays) + '日'


        checkdit[colNames[0]] = selecttype
        checkdit[colNames[1]] = selectdatef
        checkdit[colNames[2]] = stockno
        checkdit[colNames[3]] = stockname
        checkdit[colNames[4]] = price
        checkdit[colNames[5]] = buydatef
        checkdit[colNames[6]] = buyprice
        checkdit[colNames[7]] = selldatef
        checkdit[colNames[8]] = sellprice
        checkdit[colNames[9]] = diff
        checkdit[colNames[10]] = updown
        checkdit[colNames[11]] = caltype
        checkdit[colNames[12]] = updatecaldays


        '''
        checkdit['選股類型'] = '系統-決策樹'
        checkdit['選股日'] = selectdatef
        checkdit['股票代號'] = stockno
        checkdit['股票名稱'] = stockname
        checkdit['收盤價'] = price
        checkdit['買入日'] = buydatef
        checkdit['買入收盤價'] = buyprice
        checkdit['賣出日'] = selldatef
        checkdit['賣出收盤價'] = sellprice
        checkdit['漲幅'] = diff
        checkdit['賺'] = updown
        checkdit['計算類型'] = '5日'
        checkdit['計算天數'] = 5
        '''
        checklist.append(checkdit)
    
    
    checkdf = pd.DataFrame(checklist,columns=colNames)
    return checkdf


# main -----------------------------------
weekmask = [1,1,1,1,1,0,0]
holidays = ['2018-04-04','2018-04-05','2018-04-06','2018-05-01','2018-06-18']
getdate = '2018-07-27'


#dfsys = pd.read_excel("c:/pythondata/StockSystem_20180726_系統選股.xlsx")
#dfsys = pd.read_excel("c:/pythondata/StockSystem_20180418_0601_系統選股2.xlsx")
#dfsys = pd.read_excel("c:/pythondata/StockMerg_20180418_0601 _多.xlsx")
#dfsys = dfsys[dfsys['上市上櫃'] == 1]

con = sqlite3.connect("stock.db")
dfsys = pd.read_sql('select * from investor where 外資連N日買超 > 0 and 交易日 >= "2018-06-01" and 外資買超排行 <=30',con)
print('read sys select ok')

dfinv = pd.read_sql('select * from investor',con)
dfinv = dfinv.loc[:,['交易日','證券代號','證券名稱','外資連N日買超','外資買超排行']]
print('read stock investor ok')


dfprice = pd.read_sql('select * from price',con)
dfprice = dfprice.loc[:,['交易日','證券代號','證券名稱','收盤價']]

print('read stock price ok')

list_df = []

#caldayslist = [1,2,3,4,5,10,15,20]
caldayslist = [1,5,10,15,20]

'''
#by type: 全, check days,today,sell
for d in caldayslist:
    getdf1 = checkScore(dfp,selecttype='聰全過濾',checktype='days',caldays=d)
    list_df.append(getdf1)
    #getdf.head()
    

# today
getdf2 = checkScore(dfp,selecttype='聰全過濾',checktype='today',endday=getdate)
list_df.append(getdf2)
#getdf2.head()

# empty
getdf3 = checkScore(dfp,selecttype='聰全過濾',checktype='sell',endday=getdate)
list_df.append(getdf3)
'''


#by type, check days,today,sell
selecttype2 = '系統-選股'

for d1 in caldayslist:
    getdf1 = checkScore(dfsys,selecttype=selecttype2,checktype='days',caldays=d1)
    list_df.append(getdf1)
    #getdf.head()
    

'''    
# today
getdf2 = checkScore(dfsys,selecttype=selecttype2,checktype='today',endday=getdate)
list_df.append(getdf2)
#getdf2.head()


# sell
getdf3 = checkScore(dfsys,selecttype=selecttype2,checktype='sell',endday=getdate)
#getdf2.head()
list_df.append(getdf3)
'''

# save to file
dfw1 = pd.concat(list_df)
writer1= pd.ExcelWriter("c:/pythondata/checkSocre_20180607_0726_確認漲幅_外資連買選股.xlsx")
#writer1= pd.ExcelWriter("c:/pythondata/checkSocre_20180418_0601_0615_2.xlsx")
#writer1= pd.ExcelWriter("c:/pythondata/StockMerg_20180521_0525.xlsx")
dfw1.to_excel(writer1,'Sheet1',index = False)
writer1.save() 
print('save ok')

read sys select ok
read stock investor ok
read stock price ok
checktype: days
caldays: 1
checktype: days
caldays: 5
checktype: days
caldays: 10
checktype: days
caldays: 15
checktype: days
caldays: 20
save ok


In [23]:
dfan = pd.read_excel("c:/pythondata/checkSocre_20180607_0726_確認漲幅_外資連買選股.xlsx")
dfan.head()
dfan = dfan[dfan['計算天數'] > 0]
dfan['win'] = dfan['賺']
dfan['type'] = dfan['計算類型']
dfan.head()
dfan.loc[dfan['win'] != 'Y','win'] = 0
dfan.loc[dfan['win'] == 'Y','win'] = 1
dfan['漲幅'] = dfan['漲幅'].astype(float)
dfan.pivot_table('win', index='選股類型',columns='type')

type,10日,15日,1日,20日,5日
選股類型,,,,,
系統-選股,0.297701,0.268056,0.229148,0.219298,0.291462


In [24]:

y = dfsys.merge(dfan,left_on=['交易日','證券代號'],right_on=['選股日','股票代號'],how='left')

writer2= pd.ExcelWriter("c:/pythondata/checkSocre_20180607_0726_確認漲幅_外資連買選股_merg.xlsx")
#writer1= pd.ExcelWriter("c:/pythondata/StockMerg_20180521_0525.xlsx")
y.to_excel(writer2,'Sheet1',index = False)
writer2.save() 
print('save ok')

save ok


In [18]:
dfsys = pd.read_sql('select * from investor where 外資連N日買超 > 0 and 交易日 == "2018-04-20" and 外資買超排行 <=50',con)
print('read sys select ok')

read sys select ok


In [19]:
len(dfsys)

50

In [13]:
dfsys.sort_values('外資連N日買超')

,交易日,證券代號,證券名稱,外資買賣超張數,外資連N日買超,外資買超排行,外資賣超排行,投信買賣超張數,投信連N日買超,投信買超排行,投信賣超排行,自營商買賣超張數,自營商連N日買超,自營商買超排行,自營商賣超排行,上市上櫃
23811,2018-05-16,6155,鈞寶,6,1,344,594,0,0,425,513,0,0,441,497,1
8988,2018-07-03,1587,吉茂,1,1,261,707,0,0,477,491,0,0,433,535,1
8990,2018-07-03,4930,燦星網,1,1,261,707,0,0,477,491,-1,-1,650,318,1
8991,2018-07-03,00699R,元大美債7-10反1,50,1,127,841,0,0,477,491,-50,-1,842,126,1
8992,2018-07-03,00710B,FH彭博高收益債,6900,1,2,966,-6900,-1,967,1,0,0,433,535,1
8993,2018-07-03,00712,FH富時不動產,9,1,207,761,0,0,477,491,-9,-1,749,218,1
15848,2018-07-25,9908,大台北,2,1,384,527,0,0,501,410,1,1,218,693,1
15847,2018-07-25,2509,全坤建,3,1,366,544,0,0,501,410,0,0,380,530,1
15846,2018-07-25,1443,立益,3,1,366,544,0,0,501,410,0,0,380,530,1
23981,2018-05-17,2388,威盛,529,1,50,873,0,0,462,461,0,0,430,493,1
